要用 JScript 建置一個使用 ADODB.Connection 的物件導向 function 庫，並且能夠進行新增、修改、刪除、查詢操作，同時支持批次操作，以下是一個基本的實現範例。

假設你的環境支持 ActiveX 物件，以下是具體步驟：

設置全域變數 ConnectionString
創建 Database 類別
創建 executeQuery、executeNonQuery 和批次操作方法
使用 Watch.Log 進行日誌顯示
首先，設定全域變數 ConnectionString：

In [ ]:
var ConnectionString = "Your Connection String Here";


接下來，創建 Database 類別：

In [ ]:
function Database() {
    this.connection = new ActiveXObject("ADODB.Connection");
    this.connection.ConnectionString = ConnectionString;
    this.connection.Open();
}

Database.prototype.close = function() {
    this.connection.Close();
};

Database.prototype.log = function(message, level) {
    Watch.Log(message, level);
};


創建新增、修改、刪除、查詢的方法：

In [ ]:
Database.prototype.executeQuery = function(query) {
    var rs = new ActiveXObject("ADODB.Recordset");
    rs.Open(query, this.connection);
    var results = [];
    while (!rs.EOF) {
        var result = {};
        for (var i = 0; i < rs.Fields.Count; i++) {
            result[rs.Fields(i).Name] = rs.Fields(i).Value;
        }
        results.push(result);
        rs.MoveNext();
    }
    rs.Close();
    this.log("Query executed: " + query, 3);
    return results;
};

Database.prototype.executeNonQuery = function(query) {
    var cmd = new ActiveXObject("ADODB.Command");
    cmd.ActiveConnection = this.connection;
    cmd.CommandText = query;
    cmd.Execute();
    this.log("NonQuery executed: " + query, 3);
};

Database.prototype.insert = function(table, data) {
    var keys = [], values = [];
    for (var key in data) {
        keys.push(key);
        values.push("'" + data[key] + "'");
    }
    var query = "INSERT INTO " + table + " (" + keys.join(", ") + ") VALUES (" + values.join(", ") + ")";
    this.executeNonQuery(query);
};

Database.prototype.update = function(table, data, where) {
    var set = [];
    for (var key in data) {
        set.push(key + "='" + data[key] + "'");
    }
    var query = "UPDATE " + table + " SET " + set.join(", ") + " WHERE " + where;
    this.executeNonQuery(query);
};

Database.prototype.delete = function(table, where) {
    var query = "DELETE FROM " + table + " WHERE " + where;
    this.executeNonQuery(query);
};


創建批次操作的方法：

In [ ]:
Database.prototype.batchInsert = function(table, dataArray) {
    for (var i = 0; i < dataArray.length; i++) {
        this.insert(table, dataArray[i]);
    }
};

Database.prototype.batchUpdate = function(table, dataArray, whereArray) {
    for (var i = 0; i < dataArray.length; i++) {
        this.update(table, dataArray[i], whereArray[i]);
    }
};

Database.prototype.batchDelete = function(table, whereArray) {
    for (var i = 0; i < whereArray.length; i++) {
        this.delete(table, whereArray[i]);
    }
};


In [ ]:
var ConnectionString = "Your Connection String Here";

function Database() {
    this.connection = new ActiveXObject("ADODB.Connection");
    this.connection.ConnectionString = ConnectionString;
    this.connection.Open();
}

Database.prototype.close = function() {
    this.connection.Close();
};

Database.prototype.log = function(message, level) {
    Watch.Log(message, level);
};

Database.prototype.executeQuery = function(query) {
    var rs = new ActiveXObject("ADODB.Recordset");
    rs.Open(query, this.connection);
    var results = [];
    while (!rs.EOF) {
        var result = {};
        for (var i = 0; i < rs.Fields.Count; i++) {
            result[rs.Fields(i).Name] = rs.Fields(i).Value;
        }
        results.push(result);
        rs.MoveNext();
    }
    rs.Close();
    this.log("Query executed: " + query, 3);
    return results;
};

Database.prototype.executeNonQuery = function(query) {
    var cmd = new ActiveXObject("ADODB.Command");
    cmd.ActiveConnection = this.connection;
    cmd.CommandText = query;
    cmd.Execute();
    this.log("NonQuery executed: " + query, 3);
};

Database.prototype.insert = function(table, data) {
    var keys = [], values = [];
    for (var key in data) {
        keys.push(key);
        values.push("'" + data[key] + "'");
    }
    var query = "INSERT INTO " + table + " (" + keys.join(", ") + ") VALUES (" + values.join(", ") + ")";
    this.executeNonQuery(query);
};

Database.prototype.update = function(table, data, where) {
    var set = [];
    for (var key in data) {
        set.push(key + "='" + data[key] + "'");
    }
    var query = "UPDATE " + table + " SET " + set.join(", ") + " WHERE " + where;
    this.executeNonQuery(query);
};

Database.prototype.delete = function(table, where) {
    var query = "DELETE FROM " + table + " WHERE " + where;
    this.executeNonQuery(query);
};

Database.prototype.batchInsert = function(table, dataArray) {
    for (var i = 0; i < dataArray.length; i++) {
        this.insert(table, dataArray[i]);
    }
};

Database.prototype.batchUpdate = function(table, dataArray, whereArray) {
    for (var i = 0; i < dataArray.length; i++) {
        this.update(table, dataArray[i], whereArray[i]);
    }
};

Database.prototype.batchDelete = function(table, whereArray) {
    for (var i = 0; i < whereArray.length; i++) {
        this.delete(table, whereArray[i]);
    }
};
